# Load Modules

In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2

# Load Source list files

In [2]:
src_list = pd.read_csv('../source_list/cv_segmented_list.txt' , header=None , names = ['fname'] , na_values='NaN')['fname'].to_list()
src_list
source = pd.DataFrame()
for s in src_list[:]:
    temp = pd.read_csv('../source_list/cv_updated/good/'+s).iloc[:,1:]
    source = source.append(temp)
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
cv_class = ['N', 'Candidate_CV', 'CataclyV', 'Candidate_Nova', 'Candidate',
       'DN', 'known_CV', 'Cataclysmic_Variable', 'Known_CV']
cv_class = [
    'Candidate_CV', 'CataclyV', 'Candidate_Nova', 'Candidate',
       'DN', 'known_CV', 'Cataclysmic_Variable', 'Known_CV'
] 
source = source[source['Type'].isin(cv_class)].reset_index(drop=True)
display(source)

,Name,Disc. Date,Mag.,R.A.,Dec.,num_obs,Type
0,CSS120424:125906+242634,2012/04/24,NaN,12:59:05.78,+24:26:34.4,572,Candidate_CV
1,W32,NaN,NaN,00:24:05.716,-72:04:49.44,556,CataclyV
2,CXOGlb_J002405.2-720446,NaN,NaN,00:24:05.216,-72:04:46.59,523,CataclyV
3,Cl__NGC_104_MAM_M54898,NaN,NaN,00:24:06.013,-72:04:56.20,523,CataclyV
4,CXOGlb_J002404.9-720455,NaN,NaN,00:24:04.913,-72:04:55.36,478,CataclyV
...,...,...,...,...,...,...,...
309,N2403_2008-01a,NaN,NaN,07:37:01.33,+65:35:14.7,1,Candidate_Nova
310,MASTER_OT_JJ040556.85+714813.1,2013/Dec/23,18.2,04:05:56.85,+71:48:13.1,1,Known_CV
311,CX25,NaN,NaN,19:10:51.957,-59:58:40.55,1,Candidate_CV
312,MASTER_OT_JJ105545.20+573109.7,2014/Mar/04,15.5,10:55:45.20,+57:31:09.7,1,Known_CV


In [3]:
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
display(source)

,Name,Disc. Date,Mag.,R.A.,Dec.,num_obs,Type
0,CSS120424:125906+242634,2012/04/24,NaN,12:59:05.78,+24:26:34.4,572,Candidate_CV
1,W32,NaN,NaN,00:24:05.716,-72:04:49.44,556,CataclyV
2,CXOGlb_J002405.2-720446,NaN,NaN,00:24:05.216,-72:04:46.59,523,CataclyV
3,Cl__NGC_104_MAM_M54898,NaN,NaN,00:24:06.013,-72:04:56.20,523,CataclyV
4,CXOGlb_J002404.9-720455,NaN,NaN,00:24:04.913,-72:04:55.36,478,CataclyV
...,...,...,...,...,...,...,...
309,USNO-B1.0_1021-00697450,NaN,NaN,20:34:21.8666,+12:06:56.943,1,CataclyV
310,USNO-A2.0_1350-05105569,NaN,NaN,04:57:06.9743,+45:27:48.556,1,CataclyV
311,N4374_2011-04c,NaN,NaN,12:25:03.5,+12:52:18,1,Candidate_Nova
312,Gaia20dfu,2020,13.78,16:02:33.6,-60:32:50.28,1,known_CV


# Read Chandra data files 

In [4]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['Name'][index]
    df = pd.read_csv('../data/all_data/CV_v2_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    #df_select = df_select.drop(columns=['name'])
    #display(df_select.head())
    #break
    n_obs_cur = len(df_select)
    obs_id = ['CV_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['CV_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    df_select.insert(3 , 'src_n' , [s]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('CV_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
source_updated = source.reset_index(drop=True)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 314/314 [00:32<00:00,  9.77it/s]


,Name,Disc. Date,Mag.,R.A.,Dec.,num_obs,Type
0,CSS120424:125906+242634,2012/04/24,NaN,12:59:05.78,+24:26:34.4,572,Candidate_CV
1,W32,NaN,NaN,00:24:05.716,-72:04:49.44,556,CataclyV
2,CXOGlb_J002405.2-720446,NaN,NaN,00:24:05.216,-72:04:46.59,523,CataclyV
3,Cl__NGC_104_MAM_M54898,NaN,NaN,00:24:06.013,-72:04:56.20,523,CataclyV
4,CXOGlb_J002404.9-720455,NaN,NaN,00:24:04.913,-72:04:55.36,478,CataclyV
...,...,...,...,...,...,...,...
309,USNO-B1.0_1021-00697450,NaN,NaN,20:34:21.8666,+12:06:56.943,1,CataclyV
310,USNO-A2.0_1350-05105569,NaN,NaN,04:57:06.9743,+45:27:48.556,1,CataclyV
311,N4374_2011-04c,NaN,NaN,12:25:03.5,+12:52:18,1,Candidate_Nova
312,Gaia20dfu,2020,13.78,16:02:33.6,-60:32:50.28,1,known_CV


,src_id,num_obs,src_n,sepn,instrument,detect_stack_id,name,ra,dec,obsid,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,CSS120424:125906+242634,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,10553,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_1,CV_0,25,CSS120424:125906+242634,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14931,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_2,CV_0,25,CSS120424:125906+242634,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14930,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_3,CV_0,25,CSS120424:125906+242634,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14929,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_4,CV_0,25,CSS120424:125906+242634,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14928,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,USNO-B1.0_1021-00697450,0.877141,ACIS,acisfJ1743317m271131_001,2CXO J174311.1-271622,265.79630,-27.27295,8658,...,NaN,NaN,NaN,NaN,2,0,0,0,0,0
CV_310_obs_0,CV_310,1,USNO-A2.0_1350-05105569,14.232090,ACIS,acisfJ0457051p452725_001,2CXO J045708.3+452749,74.28467,45.46383,3878,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_311_obs_0,CV_311,1,N4374_2011-04c,4.922801,ACIS,acisfJ1225019p125247_001,2CXO J122503.5+125222,186.26500,12.87298,803,...,5.198e-07,NaN,NaN,NaN,5,0,0,0,0,0


## Save raw table

In [5]:
df_combined.replace(np.nan , '')
df_combined.to_csv('raw_data/cv_new_raw_data.csv' ,)

## Filter flags

In [6]:
from features import col_to_drop

In [7]:
df = pd.read_csv('raw_data/cv_new_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
print('Number of sources , obs after flux filter : {} , {} '.format(len(df['src_id'].unique()) , len(df)))
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
#display(df)
df = df.drop(columns=col_to_drop)
print('Number of sources , obs after flux filter : {} , {} '.format(len(df['src_id'].unique()) , len(df)))

/home/kumaran/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (28,231) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


BEFORE FILTERING
Number of sources , obs after flux filter : 314 , 2125 
AFTER FILTERING
Number of sources , obs after flux filter : 301 , 2101 


In [8]:
from features import arranged_cols  , id_cols 

In [9]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,streak_src_flag,likelihood_b,likelihood_h,likelihood_m,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,0.000000e+00,0.000000e+00,2.001000e-06,0.000000e+00,0.000000e+00,2.726000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_1,0.000000e+00,0.000000e+00,3.662000e-06,0.000000e+00,0.000000e+00,5.441000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_2,0.000000e+00,0.000000e+00,2.274000e-06,0.000000e+00,0.000000e+00,2.977000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_3,1.549000e-06,0.000000e+00,3.098000e-06,3.096000e-15,0.000000e+00,6.028000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_4,0.000000e+00,0.000000e+00,1.748000e-06,0.000000e+00,0.000000e+00,2.380000e-15,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_307_obs_0,7.557000e-06,4.435000e-06,1.051000e-05,1.102000e-14,6.368000e-15,1.568000e-14,False,117.40,53.39,32.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_309_obs_0,0.000000e+00,0.000000e+00,3.235000e-06,0.000000e+00,0.000000e+00,4.406000e-15,False,20.89,10.64,NaN,...,4.404000e-06,5.0,0.802,1.246000e-06,NaN,NaN,NaN,NaN,NaN,NaN
CV_311_obs_0,4.380000e-07,1.923000e-07,6.837000e-07,6.239000e-16,2.652000e-16,9.827000e-16,False,13.80,NaN,NaN,...,1.929000e-07,NaN,NaN,NaN,8.0,1.000,5.198000e-07,NaN,NaN,NaN


In [16]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,src_n,ra,dec,livetime,significance,likelihood,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,2CXO J004247.1+411619,CSS120424:125906+242634,10.69654,41.27215,4103.7,52.93,16618.73000,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_1,CV_0,25,0.157954,2CXO J004247.1+411619,CSS120424:125906+242634,10.69654,41.27215,3907.5,52.93,16618.73000,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_2,CV_0,25,0.157954,2CXO J004247.1+411619,CSS120424:125906+242634,10.69654,41.27215,3907.5,52.93,16618.73000,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_3,CV_0,25,0.157954,2CXO J004247.1+411619,CSS120424:125906+242634,10.69654,41.27215,3904.6,52.93,16618.73000,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_4,CV_0,25,0.157954,2CXO J004247.1+411619,CSS120424:125906+242634,10.69654,41.27215,3907.5,52.93,16618.73000,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_307_obs_0,CV_307,1,1.072148,2CXO J181429.7+612335,HS_1813+6122,273.62410,61.39317,4897.2,5.20,117.40180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_309_obs_0,CV_309,1,0.877141,2CXO J174311.1-271622,USNO-B1.0_1021-00697450,265.79630,-27.27295,2156.9,2.53,20.88602,...,4.404000e-06,5.0,0.802,1.246000e-06,NaN,NaN,NaN,NaN,NaN,NaN
CV_311_obs_0,CV_311,1,4.922801,2CXO J122503.5+125222,N4374_2011-04c,186.26500,12.87298,28473.1,2.16,13.80380,...,1.929000e-07,NaN,NaN,NaN,8.0,1.000,5.198000e-07,NaN,NaN,NaN


In [17]:
from features import phot_flux , phot_flux_hilim , phot_flux_lolim , en_flux , en_flux_hilim , en_flux_lolim

In [18]:
flux_cols =  en_flux + en_flux_hilim + en_flux_lolim 

In [22]:
from features import phot_flux , phot_flux_hilim , phot_flux_lolim , en_flux , en_flux_hilim , en_flux_lolim
flux_cols =  en_flux + en_flux_hilim + en_flux_lolim 
df_flux_filt = df_ok.copy().replace(np.nan , -998741)
for f in flux_cols:
    df_flux_filt = df_flux_filt.query(f+'<1e-12')
df_flux_filt = df_flux_filt.replace(-998741 , np.nan)
print('Number of sources , obs after flux filter : {} , {} '.format(len(df_flux_filt['src_id'].unique()) , len(df_flux_filt)))
df_flux_filt = df_flux_filt.query('likelihood > 0')
df_flux_filt = df_flux_filt.query('significance > 0')
print('Number of sources , obs after sig filter : {} , {} '.format(len(df_flux_filt['src_id'].unique()) , len(df_flux_filt)))

Number of sources , obs after flux filter : 282 , 2044 
Number of sources , obs after sig filter : 282 , 2044 


In [23]:
df_flux_filt.to_csv('filtered_data/cv_new_data_no_lolim.csv')